### 아이디어
- 1912(연속합)를 10만번 푸는 문제이다. 
- 위 문제를 활용하고자 한다면, $acc$ 이 $default$ 로 초기화 한다는 것은, 그 이전까지의 누적합을 더하는 것이 의미없다는 것을 의미한다.
  - 길이 $N$ 인 수열의 최대 연속 부분 누적합을 구할 수는 있겠는데, 시작점이 0이 아닌 경우에 대해선 아이디어가 필요하다.
  - 초기화해야 하는 지점을 $\Omicron(\log N)$ 만에 찾을 수 있지 않을까?

### 풀이
- 빨리 보길 잘한거 같다. 자력으로 절대 떠올릴 수 없을 법한 세그먼트의 사용법이 사용됐다.
- https://wbjeon2k.github.io/ps/almight/2021/09/19/BOJ16993.html

In [ ]:
import io, os
input=io.BytesIO(os.read(0,os.fstat(0).st_size)).readline

class ST:
  def __init__(self, L, f, default=0):
    self._def = default
    self._f = f
    self._len = len(L)
    self._size = _size = 1 << (self._len - 1).bit_length()

    self._L = L
    self.lL = [default] * (2 * _size) #s 부터 시작하는 구간의 최대값
    self.rL = [default] * (2 * _size) #s 까지의 구간의 최대값
    self.L = [default] * (2 * _size) #가운데를 지나는 구간의 최대값
    self.init(0, _size, 1)


  def init(self, left, right, node) :
    if right - left == 1 :
      if left < self._len :
        self.lL[node] = self.rL[node] = self.L[node] = self._L[left]
      return self.L[node]

    mid = (left + right) // 2
    linit = self.init(left, mid, node * 2)
    rinit = self.init(mid, right, node * 2 + 1)

    self.lL[node] = self._f(linit, self._L[mid] - self._L[left - 1] + rinit)
    self.L[node] = self._f(self.lL[node], self.rL[node])
    return self.L[node]  

  def query(self, s, e):
    s += self._size
    e += self._size

    l1 = l2 = l3 = r1 = r2 = r3 = self._def
    while s < e: 
      if s & 1:
        l1 = self._f(l1, self.L[s])
        l2 = self._f(self.lL[s], l2)
        l3 = self._f(self.L[s], l3)
        s += 1
      if e & 1:
        e -= 1
        r1 = self._f(self.L[e], r1)
        r2 = self._f(r2, self.rL[e])
        r3 = self._f(r3, self.L[e])
      s //= 2
      e //= 2

    return self._f(l1, l2, l3, r1, r2, r3)

def sol():
  N = int(input())
  L = list(map(int, input().split()))
  acc = [0] * (N + 1)
  for i in range(N):
    acc[i + 1] = acc[i] + L[i]
  print(acc)

  st = ST(acc + [0] * 10, f=max, default=-1e12)
  ans = []
  M = int(input())
  for i in range(M):
    s, e = map(int, input().split())
    ans.append(st.query(s, e + 1))
  
  print(*ans)

sol()